## Lectura de index

In [1]:
# para volver a cargar los index
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex, Settings
from llama_index.embeddings.openai import OpenAIEmbedding
import os
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

# api key
key = ''
with open('openai_key.txt', 'r') as t:
    key = t.read()
os.environ["OPENAI_API_KEY"] = key

# configuracion del modelo y embeding
embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.embed_model = embed_model
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.5)

# inicia el cliente
db = chromadb.PersistentClient(path="./index_data")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# lee el index del contenido almacenado
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

## Implementacion de sistema RAG

In [2]:
from llama_index.core.retrievers import VectorIndexRetriever

# definicion del Retiever considerando los 5 nodos mas similares
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,
)
#definicion del motor de busqueda
query_engine = index.as_query_engine()


In [3]:
def get_answer(query):
    
    # contexto de la respuesta
    context = "Eres un asistente legal entrenado para proporcionar información sobre la "\
    "Constitución de México. Tu tarea es responder preguntas legales específicas consultando "\
    "la Constitución y proporcionar la respuesta exacta. "

    # extraccion de informacion
    nodes = retriever.retrieve(query)
    # accede a la informacion del retrieve dado el query
    docs = ''
    for ni in nodes:
        aux_dic = dict(ni)
        #print(aux_dic)
        docs += f"DOCUMENT NAME: Document {aux_dic['node'].metadata['nombre']}  CONTENT: {aux_dic['node'].text} \n"

    ret = "Asegúrate de citar el o los artículos correspondientes de donde sacaste la "\
    f"información utilizando los siguientes documentos {docs}. "

    # few shot learning y definicion del formato de respuesta
    format = "Sé conciso pero preciso en tus respuestas. Si la pregunta se refiere a derechos "\
    "humanos, menciona la sección de la Constitución que habla sobre derechos fundamentales. "\
    "Si la pregunta es sobre procedimientos legales o derechos específicos, dirígete al artículo "\
    "relevante que trata ese tema. Formato de respuesta: Respuesta: [tu respuesta clara y concisa] "\
    "Fuente: Artículo [número del artículo] Ejemplo: Pregunta: ¿Qué derechos tienen los ciudadanos "\
    "mexicanos en cuanto a la libertad de expresión? Respuesta: Los ciudadanos mexicanos tienen "\
    "derecho a expresarse libremente, sin interferencias, siempre y cuando no se atente contra la "\
    "moral, los derechos de terceros, o provoque algún delito o disturbio. Fuente: Artículo 6 de "\
    "la Constitución de México. "

    # pregunta del usuario
    aux_query = f"La pregunta es {query}"

    # promt final
    promt = context + ret + format + aux_query

    # generacion de respuesta
    response = query_engine.query(promt)
    
    return response.response    

In [4]:
import pprint

querys = ["Cual es el conocimiento basico que debo tener como mexicano?", 'Cuales son mis derechos como trabajador?',
          "En la constitucion, que leyes cuidan la naturaleza?", "Como esta compuesta la constitucion, que temas abarca?",
          "Bajo que circunstancias me puede detener la policia?"]

for qi in querys:
    print(f'CONSULTA: {qi}')
    anws = get_answer(qi)
    pprint.pprint(f'RESPUESTA: {anws}\n')

CONSULTA: Cual es el conocimiento basico que debo tener como mexicano?
('RESPUESTA: Respuesta: Como mexicano, es fundamental conocer tus '
 'obligaciones, que incluyen asegurar que los menores asistan a la educación '
 'obligatoria, participar en la instrucción cívica y militar, alistarse en los '
 'cuerpos de reserva, y contribuir a los gastos públicos de manera '
 'proporcional. También es importante conocer tus derechos, como los derechos '
 'humanos garantizados por la Constitución. Fuente: Artículo 31 y Artículo 1o '
 'de la Constitución de México.\n')
CONSULTA: Cuales son mis derechos como trabajador?
('RESPUESTA: Como trabajador, tienes derechos fundamentales que incluyen:\n'
 '\n'
 '1. Derecho a un trabajo digno y socialmente útil.\n'
 '2. Jornada máxima de ocho horas diarias y siete horas nocturnas, con derecho '
 'a pago adicional por horas extraordinarias.\n'
 '3. Derecho a un día de descanso por cada seis días trabajados, con goce de '
 'salario íntegro.\n'
 '4. Vacaciones 